# 使用LlamaIndex + Metaphor实现知识工作自动化
参考文献：https://blog.llamaindex.ai/llamaindex-metaphor-towards-automating-knowledge-work-with-llms-5520a32efa2f  

将LlamaIndex数据代理的功能与Metaphor作为一种本地LLM搜索工具相结合，使知识工作者能够回答任何数据上的任何问题，无论是最近的还是复杂的。

- RAG通常需要两个核心组件：
    - 通用抽象，允许LLM以“读取”和“写入”的方式智能地对数据执行各种任务；（LlamaIndex）
    - 一个适合LLM使用的好搜索引擎（Metaphor）

LlamaHub提供了Metaphor API接口，包括如下5个工具可供Agent使用。
- 搜索：是Metaphor的入口——Agent可以通过自然语言向Metaphor搜索引擎进行查询。查询还可以包含一些附加参数，例如返回结果的数量、要包含/排除的领域以及日期筛选器；
- 检索文档：根据搜索到的文档内容从中检索出符合条件的部分内容；
- 搜索和检索文档：结合了“搜索”和“检索文档”的功能；
- 查找相似：直接调用Metaphor提供的端点，可以返回与给定URL相似的文档列表；
- 当前日期：这是一个返回当前日期的函数。就其本身而言，它与Metaphor的API无关，但可能会事先调用它，以确定传递到Metaphor的某些端点的正确日期过滤器。

## Metaphor工具测试

In [ ]:
# Set up Metaphor tool
from llama_hub.tools.metaphor.base import MetaphorToolSpec
metaphor_tool = MetaphorToolSpec(api_key='your-key',)
# convert tool spec to a list of tools
metaphor_tool_list = metaphor_tool.to_tool_list()
for tool in metaphor_tool_list:
    print(tool.metadata.name)

In [ ]:
metaphor_tool.search('machine learning transformers', num_results=3)

"""
[{'title': 'On the potential of Transformers in Reinforcement Learning',
'url': 'https://lorenzopieri.com/rl_transformers/',
'id': 'ysJlYSgeGW3l4zyOBoSGcg'},
{'title': 'Transformers: Attention in Disguise',
'url': 'https://www.mihaileric.com/posts/transformers-attention-in-disguise/',
'id': 'iEYMai5rS9k0hN5_BH0VZg'},
{'title': 'Transformers in Computer Vision: Farewell Convolutions!',
'url': 'https://towardsdatascience.com/transformers-in-computer-vision-farewell-convolutions-f083da6ef8ab?gi=a1d0a9a2896c',
'id': 'kX1Z89DdjSvBrH1S1XLvwg'}]
"""

## 使用Metaphor设置OpenAI Agent

In [ ]:
from llama_index.agent import OpenAIAgent
# We don't give the Agent our unwrapped retrieve document tools, instead passing the wrapped tools
agent = OpenAIAgent.from_tools(
  metaphor_tool_list,
  verbose=True,
)

In [ ]:
print(agent.chat("What are the best restaurants in toronto?"))

Metaphor工具实现细节
```
=== Calling Function ===
Calling function: search with args: {
  "query": "best restaurants in Toronto"
}
[Metaphor Tool] Autoprompt string: Here's a link to the best restaurant in Toronto:
Got output: [{'title': 'Via Allegro Ristorante - Toronto Fine Dining Restaurant', 'url': 'https://viaallegroristorante.com/', 'id': 'EVlexzJh-lzkVr4tb2y_qw'}, {'title': 'The Senator – Home', 'url': 'https://thesenator.com/', 'id': 'dA3HVr5P8E0Bs7nH2gH7ZQ'}, {'title': 'Home - The Rushton', 'url': 'https://therushton.com/', 'id': '6Je-igG-i-ApqISC5XXmGQ'}, {'title': 'Location', 'url': 'https://osteriagiulia.ca/', 'id': 'HjP5c54vqb3n3UNa3HevSA'}, {'title': 'StockYards | Stockyards Toronto', 'url': 'https://www.thestockyards.ca/', 'id': 'Pffz-DQlOepqVgKQDmW5Ig'}, {'title': 'Select A Restaurant', 'url': 'https://www.torontopho.com/', 'id': 'DiQ1hU1gmrIzpKnOaVvZmw'}, {'title': 'Home | Kit Kat Italian Bar & Grill', 'url': 'http://www.kitkattoronto.com/', 'id': 'kdAcLioBgnwzuHyd0rWS1w'}, {'title': 'La Fenice', 'url': 'https://www.lafenice.ca/', 'id': 'M-LHQZP6V40V81fqLFAQxQ'}, {'title': 'Le Phénix', 'url': 'https://www.lephenixto.com/', 'id': 'spCTcFr0GHlFUTzyngfRVw'}, {'title': 'ITALIAN, INSPIRED.', 'url': 'https://figotoronto.com/', 'id': 'OvBcTqEo1tCSywr4ATptCg'}]
========================
Here are some of the best restaurants in Toronto:

1. [Via Allegro Ristorante](https://viaallegroristorante.com/)
2. [The Senator](https://thesenator.com/)
3. [The Rushton](https://therushton.com/)
4. [Osteria Giulia](https://osteriagiulia.ca/)
5. [Stockyards](https://www.thestockyards.ca/)
6. [Toronto Pho](https://www.torontopho.com/)
7. [Kit Kat Italian Bar & Grill](http://www.kitkattoronto.com/)
8. [La Fenice](https://www.lafenice.ca/)
9. [Le Phénix](https://www.lephenixto.com/)
10. [Figo](https://figotoronto.com/)

You can visit their websites for more information. Enjoy your dining experience in Toronto!
```

In [ ]:
print(agent.chat('tell me more about Osteria Giulia'))

"""
=== Calling Function ===
Calling function: retrieve_documents with args: {
"ids": ["HjP5c54vqb3n3UNa3HevSA"]
}
Got output: […]
========================
Osteria Giulia is a restaurant located at 134 Avenue Road in Toronto, Ontario. You can contact them at 416.964.8686 or via email at info@osteriagiulia.ca (for general inquiries only, no reservation requests via email).
The restaurant's operating hours are from Monday to Saturday, from 5:00pm to 11:00pm. On Sundays, the restaurant is available for private bookings.
Parking is available on Avenue Road and Davenport Road.
You can follow Osteria Giulia on Instagram [@osteriagiulia](https://www.instagram.com/osteriagiulia). They also have a sister restaurant called Giulietta, which you can visit at [giu.ca](https://giu.ca) or on Instagram [@giulietta972](https://www.instagram.com/giulietta972).
Please note that the information provided is based on the available document and may be subject to change. It is recommended to visit their official website or contact them directly for the most up-to-date information.
"""

## 避免上下文窗口问题（高级）

       使用retrieve的一个问题是内容可能很长。如果内容被直接地附加到会话历史并转储到LLM上下文窗口中，那么我们可能会遇到上下文窗口限制。

    LlamaIndex提供了工具抽象来帮助处理这一问题。我们的LoadAndSearchToolSpec嵌入了任何可能返回大量数据的工具，并将其分为两个工具：一个是将数据动态存储在索引中的加载工具，另一个是允许在该索引上进行搜索的搜索工具。

       在Metaphor方面，我们定义search_and_recovere_documents端点来结合search和retrieve。这允许代理进行单个查询以检索大量文档，当这些文档与LoadAndSearchToolSpec结合使用时，这些文档将直接存储在索引中。如果代理分别调用search和retrieve，那么将搜索结果写入会话历史记录，然后再次将其传递到提示中，以调用retrieve覆盖所有文档ID，将花费更长的时间，并消耗更多的token。

       创建LoadAndSearchToolSpec：

In [ ]:
from llama_index.tools.tool_spec.load_and_search.base import LoadAndSearchToolSpec
# The search_and_retrieve_documents tool is the third in the tool list, as seen above
wrapped_retrieve = LoadAndSearchToolSpec.from_defaults(
  metaphor_tool_list[2],
)

In [ ]:
# Just pass the wrapped tools and the get_date utility
agent = OpenAIAgent.from_tools(
  [*wrapped_retrieve.to_tool_list(), metaphor_tool_list[4]],
  verbose=True,
)
print(agent.chat('Can you summarize everything published in the last month regarding news on superconductors'))


"""
=== Calling Function ===
Calling function: current_date with args: {}
Got output: 2023-08-20
========================
=== Calling Function ===
Calling function: search_and_retrieve_documents with args: {
  "query": "superconductors",
  "start_published_date": "2023-07-20",
  "end_published_date": "2023-08-20"
}
[Metaphor Tool] Autoprompt: "Here is an interesting article about superconductors:
Got output: Content loaded! You can now search the information using read_search_and_retrieve_documents
========================
=== Calling Function ===
Calling function: read_search_and_retrieve_documents with args: {
  "query": "superconductors"
}
Got output: 
Superconductors are materials that can perfectly conduct electricity. They are used in a variety of applications, such as particle accelerators, nuclear fusion devices, MRI machines, and maglev trains. However, so far, no superconductor has been proven to work at ambient pressures and temperatures. On July 22, scientists in South Korea published research claiming to have solved this problem with a material called LK-99, which has an electrical resistivity that drops to near zero at 30 degrees Celsius (86 degrees Fahrenheit).
========================
In the last month, there have been developments in the field of superconductors. Scientists in South Korea have published research on a material called LK-99, which has the ability to conduct electricity with near-zero resistance at a temperature of 30 degrees Celsius (86 degrees Fahrenheit). This breakthrough could potentially lead to the development of superconductors that work at ambient pressures and temperatures, opening up new possibilities for various applications such as particle accelerators, nuclear fusion devices, MRI machines, and maglev trains.
"""